In [38]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [39]:
import ipynb.fs.full.TrainTest as TrainTest

p = 30 # want this to be high
w = 4  # want this to be low
cw = 9 # want this to be high
act = [3]
# act_string = ''
act_string = '_a' + str(act)
nperseg = 2
# data = np.load('./data/running/nperseg' + str(nperseg) + '/' + 'p' + str(p) + act_string + '_w' + str(w) + '_cw' + str(cw) + '.npy')

data = np.load('./data/running/nperseg2/p30_a[2, 3]_w4_cw9.npy')


x_train, y_train, x_test, y_test = TrainTest.get_train_test(data, 0.8)

Do RandomizedSearchCV

In [40]:
def getBestParams(clf, param_options):
    clf_grid = GridSearchCV(estimator = clf, param_grid = param_options, cv = 3, verbose=1, n_jobs = -1)
    clf_grid.fit(x_train, y_train)
    return clf_grid.best_params_

In [41]:
from sklearn.metrics import accuracy_score, precision_score, f1_score

# options for nearest neighbors
param_options = {
    'n_neighbors' : range(1, 6),
    'weights' : ['uniform', 'distance'],
    'algorithm' : ['auto'],
    'leaf_size' : [10, 20, 30, 40, 50],
    'p' : [1, 2, 3, 4],
    'metric' : ['euclidean', 'manhattan', 'chebyshev', 'minkowski'],
#     'metric_params' : [],
#     'n_jobs' : []
}

knn = KNeighborsClassifier(**getBestParams(KNeighborsClassifier(), param_options))

avg_acc = 0
avg_prec = 0
avg_f1 = 0
num = 10
for i in range(num):
    knn.fit(x_train, y_train)
    
    y_pred = knn.predict(x_test)
    
    avg_acc += accuracy_score(y_test, y_pred)
    avg_prec += precision_score(y_test, y_pred)
    avg_f1 += f1_score(y_test, y_pred)

avg_acc /= num
avg_prec /= num
avg_f1 /= num

print('Average accuracy:  ' + str(avg_acc))
print('Average precision: ' + str(avg_prec))
print('Average F1 Score:  ' + str(avg_f1))

Fitting 3 folds for each of 800 candidates, totalling 2400 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   34.0s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 2400 out of 2400 | elapsed:  8.2min finished


Average accuracy:  0.6932203389830508
Average precision: 0.6618075801749271
Average F1 Score:  0.7149606299212597
